In [ ]:
# system
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# fastAI
from fastai.vision.all import *
from fastai.callback.tracker import Recorder
from fastai.interpret import ClassificationInterpretation

import matplotlib.pyplot as plt

In [ ]:
from pathlib import Path

# If using the current directory
current_dir = Path.cwd()

# If you need to navigate relative to the current directory
project_root = current_dir.parent.parent.parent
relative_path = project_root / 'dataset' / 'rotation' / 'train'


In [ ]:
aug_transforms_list = aug_transforms(size=224)

# Define data loading and preprocessing pipeline using DataBlock
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                get_items=get_image_files,
                splitter=RandomSplitter(valid_pct=0.2, seed=42),
                get_y=parent_label,
                item_tfms=Resize(224), 
                batch_tfms=aug_transforms_list)

dls = dblock.dataloaders(relative_path, bs=32, num_workers=0)

In [ ]:
# Create a learner with ResNet18 architecture
epochs = 50

learn = vision_learner(dls, resnet50, metrics=accuracy, ps=0, wd=0)

# Attach Recorder callback to track metrics during training
recorder = Recorder()
learn.add_cb(recorder)

# Fine-tune the model for 2 epochs with a base learning rate of 3e-3
learn.fine_tune(epochs, base_lr=3e-3)

path = os.path.join('./', f"test.pkl")

learn.export(path)
# Plot the training and validation losses

recorder.plot_loss()

In [ ]:
from fastai.vision.all import load_learner

# Load the trained model
path = './test.pkl'
learn = load_learner(path)

# Load an image
img_path = 'test.jpg'
img = Image.open(img_path)
img = img.rotate(-260)
# If you used any specific item transforms during training, apply them here
img = img.resize((224, 224))  # Example resize, adjust according to your training setup

plt.imshow(img)

# Make a prediction
pred, pred_idx, probs = learn.predict(img)

print(f"Prediction: {pred}")
print(f"Prediction Index: {pred_idx}")
print(f"Probabilities: {probs}")


In [ ]:
from fastai.vision.all import *
import timm

def timm_vision_transformer(name, pretrained=True, **kwargs):
    """ Create a vision transformer model from timm library """
    return timm.create_model(name, pretrained=pretrained, **kwargs)

# Example of using a Vision Transformer model 'vit_base_patch16_224'
vit_model = timm_vision_transformer('vit_base_patch16_224', pretrained=True)

# You can now integrate this model into a fastai Learner
dls = ImageDataLoaders.from_folder(relative_path, valid_pct=0.2, item_tfms=Resize(224), batch_tfms=Normalize.from_stats(*imagenet_stats))
learn = Learner(dls, vit_model, metrics=accuracy)

# Train the model
learn.fine_tune(50)

In [ ]:
path = os.path.join('./', f"vit-3.pkl")

learn.export(path)
# Plot the training an

In [ ]:
from fastai.vision.all import load_learner

# Load the trained model
path = './vit-2.pkl'
learn = load_learner(path)

# Load an image
img_path = 'tshirt_20230920154938.png'
img = Image.open(img_path)
img = img.rotate(-275)
# If you used any specific item transforms during training, apply them here
img = img.resize((224, 224))  # Example resize, adjust according to your training setup

plt.imshow(img)

# Make a prediction
pred, pred_idx, probs = learn.predict(img)

print(f"Prediction: {pred}")
print(f"Prediction Index: {pred_idx}")
